<strong>Performance Analysis of DOA algorithms</strong>

In [ ]:
from signal_model import FarField1DSource, generate_random_angles
from doa_algorithms import Music, Capon, CramerRaoBound, Esprit, RootMUSIC, SpectrumPeakFinder
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
N_ANTENNA = 16
N_SAMPLES = 128
N_TARGETS = 2
FREQ = 1e9
COHERENT_SOURCES = False
BASE_BAND_SIGNAL = False
ANGLES = (-np.pi/3, np.pi/3, 1024)
MIN_ANGLE_RES = np.deg2rad(5)
ANGLES_RANGE = np.linspace(ANGLES[0], ANGLES[1], ANGLES[2], endpoint=False)
MIN_ANGLE_DIS = int(MIN_ANGLE_RES//(ANGLES_RANGE[1]-ANGLES_RANGE[0]))
ANGLES_RANGE_TEST = ANGLES_RANGE[0:-MIN_ANGLE_DIS]
SNR = np.arange(-20, 21, 5)
T = np.arange(25, 225, 25)
NUM_ITER = 50

source = FarField1DSource(N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)
music_ = Music(ANGLES_RANGE, None, N_ANTENNA, FREQ)
crb_ = CramerRaoBound(N_SAMPLES,  N_ANTENNA, FREQ)
rootmusic_ = RootMUSIC(N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)
esprit_ = Esprit(N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)
capon_ = Capon(ANGLES_RANGE, N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)

peak_finder = SpectrumPeakFinder(
    expected_peaks=N_TARGETS,
    filter_type='butterworth'
)


<strong>RMSE vs. SNR</strong>

In [ ]:
MUSIC = np.zeros((SNR.shape[0], ), dtype=np.float32)
ROOTMUSIC = np.zeros((SNR.shape[0], ), dtype=np.float32)
CRB = np.zeros((SNR.shape[0], ), dtype=np.float32)
CAPON = np.zeros((SNR.shape[0], ), dtype=np.float32)
ESPRIT = np.zeros((SNR.shape[0], ), dtype=np.float32)

for i, snr in enumerate(SNR):
    music_error = 0
    rootmusic_error = 0
    crb = 0
    capon_error = 0
    esprit_error = 0

    for _ in range(NUM_ITER):
        angle = np.sort(generate_random_angles(N_TARGETS, ANGLES_RANGE, MIN_ANGLE_RES))
        sig = source.collect_plane_wave_response(angle, snr)

        music = music_.estimate(sig, N_TARGETS)
        music = peak_finder.find_peak_indices(music)
        est_angle = np.sort(ANGLES_RANGE[music])
        music_error += np.sum((est_angle - angle)**2)

        rmusic = np.sort(rootmusic_.estimate(sig))
        rootmusic_error += np.sum((rmusic - angle)**2)

        crb += crb_.crb_stochastic(angle, snr)

        capon = capon_.estimate(angle, snr)
        capon = peak_finder.find_peak_indices(capon)
        est_angle = np.sort(ANGLES_RANGE[capon])
        capon_error += np.sum((est_angle - angle)**2)

        esprit = np.sort(esprit_.estimate(angle, snr))
        esprit_error += np.sum((esprit - angle)**2)

    MUSIC[i] = np.sqrt(music_error.item()/NUM_ITER)
    ROOTMUSIC[i] = np.sqrt(rootmusic_error.item()/NUM_ITER)
    CRB[i] = np.sqrt(np.real(crb.item())/NUM_ITER)
    CAPON[i] = np.sqrt(capon_error.item()/NUM_ITER)
    ESPRIT[i] = np.sqrt(esprit_error.item()/NUM_ITER)


In [ ]:
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 14})
plt.semilogy(SNR, MUSIC, 'b^-', linewidth=2, label='MUISC')
plt.semilogy(SNR, ROOTMUSIC, 'ys-', linewidth=2, label='Root-MUSIC')
plt.semilogy(SNR, CAPON, 'r.-', linewidth=2, label='CAPON')
plt.semilogy(SNR, ESPRIT, 'g<-', linewidth=2, label='ESPRIT')
plt.semilogy(SNR, CRB, 'k--', linewidth=2, label='CRB')
plt.xlabel('SNR [dB]')
plt.ylabel('RMSE [rad]')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


<strong>RMSE vs. N_SNAPSHOTS</strong>

In [ ]:
SNR_ = 10

MUSIC = np.zeros((T.shape[0], ), dtype=np.float32)
CRB = np.zeros((T.shape[0], ), dtype=np.float32)
ROOTMUSIC = np.zeros((T.shape[0], ), dtype=np.float32)
CAPON = np.zeros((T.shape[0], ), dtype=np.float32)
ESPRIT = np.zeros((T.shape[0], ), dtype=np.float32)

for i, t in enumerate(T):
    source = FarField1DSource(N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)
    music_ = Music(ANGLES_RANGE, None, N_ANTENNA, FREQ)
    crb_ = CramerRaoBound(N_SAMPLES,  N_ANTENNA, FREQ)
    rootmusic_ = RootMUSIC(N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)
    esprit_ = Esprit(N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)
    capon_ = Capon(ANGLES_RANGE, N_SAMPLES, N_TARGETS, COHERENT_SOURCES, N_ANTENNA, FREQ, BASE_BAND_SIGNAL)

    music_error = 0
    rootmusic_error = 0
    crb = 0
    capon_error = 0
    esprit_error = 0

    for _ in range(NUM_ITER):
        angle = np.sort(generate_random_angles(N_TARGETS, ANGLES_RANGE, MIN_ANGLE_RES))
        sig = source.collect_plane_wave_response(angle, SNR_)

        music = music_.estimate(sig, N_TARGETS)
        music = peak_finder.find_peak_indices(music)
        est_angle = np.sort(ANGLES_RANGE[music])
        music_error += np.sum((est_angle - angle)**2)

        rmusic = np.sort(rootmusic_.estimate(sig))
        rootmusic_error += np.sum((rmusic - angle)**2)

        crb += crb_.crb_stochastic(angle, SNR_)

        capon = capon_.estimate(angle, SNR_)
        capon = peak_finder.find_peak_indices(capon)
        est_angle = np.sort(ANGLES_RANGE[capon])
        capon_error += np.sum((est_angle - angle)**2)

        esprit = np.sort(esprit_.estimate(angle, SNR_))
        esprit_error += np.sum((esprit - angle)**2)

    MUSIC[i] = np.sqrt(music_error.item()/NUM_ITER)
    ROOTMUSIC[i] = np.sqrt(rootmusic_error.item()/NUM_ITER)
    CRB[i] = np.sqrt(np.real(crb.item())/NUM_ITER)
    CAPON[i] = np.sqrt(capon_error.item()/NUM_ITER)
    ESPRIT[i] = np.sqrt(esprit_error.item()/NUM_ITER)


In [ ]:
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 14})
plt.semilogy(T, MUSIC, 'b^-', linewidth=2, label='MUISC')
plt.semilogy(T, ROOTMUSIC, 'ys-', linewidth=2, label='Root-MUSIC')
plt.semilogy(T, CAPON, 'r.-', linewidth=2, label='CAPON')
plt.semilogy(T, ESPRIT, 'g<-', linewidth=2, label='ESPRIT')
plt.semilogy(T, CRB, 'k--', linewidth=2, label='CRB')
plt.xlabel('N_STAPSHOTS')
plt.ylabel('RMSE [rad]')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


<strong>DOA [rad] vs. SAMPLE INDEX</strong>

In [ ]:
# # N_TARGETS should be 2
NUM_ITER = 10
SNR_ = 10

MUSIC = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)
ROOTMUSIC = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)
CAPON = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)
ESPRIT = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)

for i, doa in enumerate(ANGLES_RANGE_TEST):
    music_error = 0
    rootmusic_error = 0
    capon_error = 0
    esprit_error = 0
    angle = np.array([doa, doa+MIN_ANGLE_RES])

    for _ in range(NUM_ITER):
        sig = source.collect_plane_wave_response(angle, SNR_)

        music = music_.estimate(sig, N_TARGETS)
        music = peak_finder.find_peak_indices(music)
        music_error += np.sort(ANGLES_RANGE[music])

        rootmusic_error += np.sort(rootmusic_.estimate(sig))

        capon = capon_.estimate(angle, SNR_)
        capon = peak_finder.find_peak_indices(capon)
        capon_error += np.sort(ANGLES_RANGE[capon])

        esprit_error += np.sort(esprit_.estimate(angle, SNR_))

    MUSIC[i, :] = music_error/NUM_ITER
    ROOTMUSIC[i, :] = rootmusic_error/NUM_ITER
    CAPON[i, :] = capon_error/NUM_ITER
    ESPRIT[i, :] = esprit_error/NUM_ITER


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
datas = [MUSIC, ROOTMUSIC, CAPON, ESPRIT]
data_names = ['MUSIC', 'ROOTMUSIC', 'CAPON', 'ESPRIT']


for i, data in enumerate(datas):
    ax = axes[positions[i][0], positions[i][1]]
    ax.plot(np.arange(ANGLES_RANGE_TEST.shape[0]), ANGLES_RANGE_TEST, 'b', label=r'$\theta_1$')
    ax.plot(np.arange(ANGLES_RANGE_TEST.shape[0]), ANGLES_RANGE_TEST + MIN_ANGLE_RES, 'r', label=r'$\theta_2$')
    ax.plot(np.arange(ANGLES_RANGE_TEST.shape[0]), data[:, 0], 'b.', linewidth=1, label=r'$\hat{\theta}_1$')
    ax.plot(np.arange(ANGLES_RANGE_TEST.shape[0]), data[:, 1], 'r.', linewidth=1, label=r'$\hat{\theta}_2$')
    ax.set_title(data_names[i])

plt.show()


<strong>DOA [rad] vs. SAMPLE INDEX ERROR</strong>

In [ ]:
# N_TARGETS should be 2
NUM_ITER = 10
SNR_ = 10

MUSIC = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)
ROOTMUSIC = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)
CAPON = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)
ESPRIT = np.zeros((ANGLES_RANGE_TEST.shape[0], N_TARGETS), dtype=np.float32)

for i, doa in enumerate(ANGLES_RANGE_TEST):
    music_error = 0
    rootmusic_error = 0
    capon_error = 0
    esprit_error = 0
    angle = np.array([doa, doa+MIN_ANGLE_RES])

    for _ in range(NUM_ITER):
        sig = source.collect_plane_wave_response(angle, SNR_)

        music = music_.estimate(sig, N_TARGETS)
        music = peak_finder.find_peak_indices(music)
        music = np.sort(ANGLES_RANGE[music])
        music_error += music - angle

        rmusic = np.sort(rootmusic_.estimate(sig))
        rootmusic_error += rmusic - angle

        capon = capon_.estimate(angle, SNR_)
        capon = peak_finder.find_peak_indices(capon)
        capon = np.sort(ANGLES_RANGE[capon])
        capon_error += capon - angle

        esprit = np.sort(esprit_.estimate(angle, SNR_))
        esprit_error += esprit - angle

    MUSIC[i, :] = music_error/NUM_ITER
    ROOTMUSIC[i, :] = rootmusic_error/NUM_ITER
    CAPON[i, :] = capon_error/NUM_ITER
    ESPRIT[i, :] = esprit_error/NUM_ITER


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
datas = [MUSIC, ROOTMUSIC, CAPON, ESPRIT]
data_names = ['MUSIC', 'ROOTMUSIC', 'CAPON', 'ESPRIT']


for i, data in enumerate(datas):
    ax = axes[positions[i][0], positions[i][1]]
    ax.plot(np.arange(ANGLES_RANGE_TEST.shape[0]), data[:, 0], 'b.', linewidth=1, label=r'$\hat{\theta}_1$')
    ax.plot(np.arange(ANGLES_RANGE_TEST.shape[0]), data[:, 1], 'r.', linewidth=1, label=r'$\hat{\theta}_2$')
    ax.set_title(data_names[i])


plt.show()
